In [1]:
! pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train=to_categorical(y_train)
if y_test.ndim ==1: y_test=to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H-%M-%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    
    model.fit(
        X_train,
        y_train,
        batch_size = params_fit.get('batch_size', 128),
        epochs = params_fit.get('epochs',5),
        verbose = params_fit.get('verbose',1),
        validation_data = params_fit.get('validation_data',(X_train, y_train)),
        callbacks = [tensorboard_callback]
      )
    return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.6579 - accuracy: 0.2781 - val_loss: 1.1623 - val_accuracy: 0.6422
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.9253 - accuracy: 0.7025 - val_loss: 0.3474 - val_accuracy: 0.8764
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.4577 - accuracy: 0.8564 - val_loss: 0.1175 - val_accuracy: 0.9629
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2991 - accuracy: 0.9085 - val_loss: 0.0731 - val_accuracy: 0.9816
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2264 - accuracy: 0.9314 - val_loss: 0.0482 - val_accuracy: 0.9850


0.9530612244897959

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
    )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status' : STATUS_OK, 'model':model}

In [31]:
space = {
    'batch_size': hp.quniform('batch_size', 10, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 2.2735 - accuracy: 0.2500

 12/138 [=>............................]
 - ETA: 0s - loss: 0.5939 - accuracy: 0.7969

 23/138 [====>.........................]
 - ETA: 0s - loss: 0.6793 - accuracy: 0.8057

 35/138 [======>.......................]
 - ETA: 0s - loss: 0.4686 - accuracy: 0.8679

 44/138 [========>.....................]
 - ETA: 0s - loss: 0.4215 - accuracy: 0.8757

 55/138 [==========>...................]
 - ETA: 0s - loss: 0.4203 - accuracy: 0.8773
